In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 27 kB/s 
     |████████████████████████████████| 198 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1d9929340e6e9ef10839258259f47264d439ef0fc8fd07d5ecfcc031729dc2f0
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [84]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Home_work_spark')\
        .getOrCreate()
spark

In [83]:
df = spark.read.csv('owid-covid-data.csv', header = True)
df.show(1)


+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+----------+------------------+----------+-------------+-------------+--------------+---

In [81]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

# вижу что есть какие-то WORLD и тд, фильтрую на 3 строке коды, т к страны состоят из 3 букв

df.filter(f.col('date') == '2021-03-31').\
  filter(f.length(f.col('iso_code')) == 3).\
  select('iso_code','total_cases','location').\
  withColumn('persent', 
             df.filter(f.col('date') == '2021-03-31').total_cases / f.lit(df.filter(f.col('date') == '2021-03-31').agg({'total_cases':'sum'}).collect()[0][0]) * 100).\
  sort(f.col('persent').desc()).\
  select('iso_code','total_cases','persent').limit(15).show()
  
  #.repartition(1).write.csv("task1.1", header = True)


+--------+-----------+------------------+
|iso_code|total_cases|           persent|
+--------+-----------+------------------+
|     USA| 30462210.0| 7.367302066171039|
|     BRA| 12748747.0| 3.083291399875184|
|     IND| 12221665.0|2.9558163313348005|
|     FRA|  4705068.0|1.1379232563190667|
|     RUS|  4494234.0|1.0869329386822602|
|     GBR|  4359982.0| 1.054464019421721|
|     ITA|  3584899.0|0.8670097740680828|
|     TUR|  3317182.0| 0.802262271925293|
|     ESP|  3284353.0|0.7943225604096043|
|     DEU|  2843644.0|0.6877368489237937|
|     COL|  2406377.0| 0.581983587011135|
|     ARG|  2348821.0|0.5680636370889022|
|     POL|  2321717.0| 0.561508519938784|
|     MEX|  2238887.0|0.5414760393623274|
|     IRN|  1885564.0|0.4560246795323691|
+--------+-----------+------------------+



In [80]:
#Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
#(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)


# фильтрую по датам, фильтрую чтобы были только страны с кодом в 3 символа, нахожу сумму в каждой локации за неделю
# беру первые 10 стран из топа, джоин чтобы получить все строки для каждой из этих стран 

df.filter(f.col('date') >= '2021-03-24').\
filter(f.col('date') <= '2021-03-31').\
filter(f.length(f.col('iso_code')) == 3).\
groupBy('location').agg({'new_cases':'sum'}).\
sort(f.col('sum(new_cases)').desc()).\
limit(10).select('location').\
join(df.filter(f.col('date') >= '2021-03-24').\
filter(f.col('date') <= '2021-03-31').\
filter(f.length(f.col('iso_code')) == 3), on = ['location']).\
orderBy(f.col("new_cases").cast("int")).\
select('date','location','new_cases').show()

#.repartition(1).write.csv("task1.2", header = True)


+----------+---------+---------+
|      date| location|new_cases|
+----------+---------+---------+
|2021-03-28|  Germany|   1727.0|
|2021-03-28|Argentina|   7208.0|
|2021-03-25|Argentina|   8238.0|
|2021-03-24|Argentina|   8300.0|
|2021-03-29|  Ukraine|   8468.0|
|2021-03-29|   France|   9110.0|
|2021-03-30|Argentina|  10154.0|
|2021-03-29|  Germany|  10297.0|
|2021-03-27|Argentina|  10338.0|
|2021-03-30|  Ukraine|  10648.0|
|2021-03-31|  Ukraine|  11345.0|
|2021-03-28|  Ukraine|  12041.0|
|2021-03-29|    Italy|  12900.0|
|2021-03-26|Argentina|  12936.0|
|2021-03-29|Argentina|  14014.0|
|2021-03-24|  Ukraine|  14266.0|
|2021-03-30|    Italy|  16055.0|
|2021-03-31|Argentina|  16056.0|
|2021-03-25|  Ukraine|  16762.0|
|2021-03-29|   Poland|  16973.0|
+----------+---------+---------+
only showing top 20 rows



In [79]:
#Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
#(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
#(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)
from pyspark.sql import Window

df.filter('iso_code = "RUS"').\
withColumn('before', f.lag(f.col('new_cases')).over(Window.orderBy('date'))).\
withColumn('diff', f.col('new_cases') - f.col ('before')).\
select('date','before','new_cases','diff').\
orderBy('date').show()

#.repartition(1).write.csv("task1.3", header = True)

+----------+------+---------+----+
|      date|before|new_cases|diff|
+----------+------+---------+----+
|2020-01-31|  null|      2.0|null|
|2020-02-01|   2.0|      0.0|-2.0|
|2020-02-02|   0.0|      0.0| 0.0|
|2020-02-03|   0.0|      0.0| 0.0|
|2020-02-04|   0.0|      0.0| 0.0|
|2020-02-05|   0.0|      0.0| 0.0|
|2020-02-06|   0.0|      0.0| 0.0|
|2020-02-07|   0.0|      0.0| 0.0|
|2020-02-08|   0.0|      0.0| 0.0|
|2020-02-09|   0.0|      0.0| 0.0|
|2020-02-10|   0.0|      0.0| 0.0|
|2020-02-11|   0.0|      0.0| 0.0|
|2020-02-12|   0.0|      0.0| 0.0|
|2020-02-13|   0.0|      0.0| 0.0|
|2020-02-14|   0.0|      0.0| 0.0|
|2020-02-15|   0.0|      0.0| 0.0|
|2020-02-16|   0.0|      0.0| 0.0|
|2020-02-17|   0.0|      0.0| 0.0|
|2020-02-18|   0.0|      0.0| 0.0|
|2020-02-19|   0.0|      0.0| 0.0|
+----------+------+---------+----+
only showing top 20 rows

